# Toxic Femininity

Web Scraping Script for incels.is Forum Posts
This script extracts the titles, replies, and views from the incels.is forum posts under a specific category. It navigates through all pages of the category until there are no more pages left. The extracted data is then saved to an Excel file.

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://incels.is"
forum_url = "/forums/inceldom-discussion.2/?prefix_id=23"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

title_slugs = []
reply_counts = []
view_counts = []

while forum_url:
    response = requests.get(base_url + forum_url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve the webpage: {base_url + forum_url}")
        break

    soup = BeautifulSoup(response.content, 'html.parser')
    title_elements = soup.find_all('a', {'data-xf-init': 'preview-tooltip'})
    data_elements = soup.find_all('dd')

    for title in title_elements:
        preview_url = title['data-preview-url']
        title_slug = preview_url.split('/')[-2]
        title_slugs.append(title_slug)

    for i in range(0, len(data_elements), 2):
        reply_counts.append(data_elements[i].text.strip())
        view_counts.append(data_elements[i+1].text.strip())

    # Check for the "Next" button and retrieve the next page's URL
    next_button = soup.find('a', {'class': 'pageNav-jump pageNav-jump--next'})
    forum_url = next_button['href'] if next_button else None

# Convert lists to DataFrame
df = pd.DataFrame({
    'Title': title_slugs,
    'Replies': reply_counts,
    'Views': view_counts
})

# Save DataFrame to Excel
df.to_excel("titles_replies_views.xlsx", index=False)

print("Saved titles, replies, and views to titles_replies_views.xlsx")


Saved titles, replies, and views to titles_replies_views.xlsx
